In [4]:
import os
import glob
import cv2
import requests
import json
from PIL import Image
from datetime import datetime
import time
from PIL.ExifTags import TAGS, GPSTAGS
import exifread

start_time = datetime.now()



#Oil Spill Model
url = 'http://localhost:8081/'
#url = 'https://app.nanonets.com/api/v2/ObjectDetection/Model/cdae82fe-76e4-4bcd-ab81-aefd9686a711/LabelFile/'

filepath = ('C:\\Users\\sami\\Desktop\\test images\\*.*') # change this path
save_anomaly_path = 'C:\\Users\\sami\\Desktop\\test images\\leak\\'# change this path

filepath_leak = 'C:\\Users\\sami\\Desktop\\test images\\leak\\*.*'# change this path
save_none_path = 'C:\\Users\\sami\\Desktop\\test images\\noLeak\\'# change this path


name_images = []
name_leak =[]
#images_location =[]
leak_images_location=[]
geotagging = {}
leak =[]
leak_images =[]

# Extract Images Location
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")
    
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]                 

    return geotagging

def get_dms(dms):    
    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] 
    seconds = dms[2][0] / dms[2][1] 
    
    return round (degrees,2) , round (minutes,2) , round (seconds,2)

def get_coordinates_dms(geatags):
    lat = get_dms(geotags['GPSLatitude'])
    lon = get_dms(geotags['GPSLongitude'])

    return lat, geotags['GPSLatitudeRef'], lon, geotags['GPSLongitudeRef']


for bb,f1 in enumerate (glob.glob(filepath)):       
    #Extract images names
    img_name = os.path.basename(f1)   
    name_images.append(img_name)
      
    #Extract image date (Only ONE image)
    if (bb < 1):
        im = Image.open(f1)
        exif = im.getexif()
        creation_time = exif.get(36867)
        onlyDate = ('%.11s' % creation_time)
        #print(onlyDate)
        
        
    data = {'file': open(f1, 'rb')}
    
    response = requests.post(url, auth=requests.auth.HTTPBasicAuth('API_KEY', ''), files=data)
    print(response.text)
    
    img = cv2.imread(f1) # read image
    
    response_json = json.loads(response.text)
    
    prediction = response_json["result"][0]["prediction"]
    
    LEAK_MIN_THRESHOLD = 0.50 # experiment with different threshold level between 0.0-1.0
        
    for i in prediction:
        if (i['label']) == ('Oil_Spill'):
            leak.append(i['score'])
            
            if max(leak) >= LEAK_MIN_THRESHOLD:
                    
                if (i['score']) >= LEAK_MIN_THRESHOLD:    
                    img = cv2.rectangle(img,(i['xmin'],i['ymin']),(i['xmax'],i['ymax']),(0,0,255),10)
                    cv2.imwrite(save_anomaly_path + str(os.path.basename(f1)) +' leak{}.jpg'.format(bb), img)
                    
                    #Make Dictionary for images with leaks only
                    if len(leak)<2:    
                        leak_images.append(f1)
                
                    #Extract images names of leaks
                        img_name_leak = os.path.basename(f1)   
                        name_leak.append(img_name_leak)
                
                    #image loctaion
                        exif = Image.open(f1)._getexif()
                        geotags = get_geotagging(exif)
                        leak_images_location.append(get_coordinates_dms(geotags))
                   
        
            if max(leak) < LEAK_MIN_THRESHOLD:
                cv2.imwrite(save_none_path + str(os.path.basename(f1)) +' no{}.jpg'.format(bb), img)
                                     
            
    if len(leak) ==0:
        cv2.imwrite(save_none_path + str(os.path.basename(f1)) +' no{}.jpg'.format(bb), img)
                        
    leak=[]  

#predict leak images
predict_leak =[]

for cc,p1 in enumerate (glob.glob(filepath_leak)):
    predict_leak.append(p1)
print(predict_leak)


# calculate number of images
No_images = len(name_images)
    
# Generate Sequence Numbers of leak 
NO_leak =[]
for i in range(2000):
    if i < len(leak_images):
        
        NO_leak.append(i+1)



#create Dictionary for leak images
#Take Info from User
area = list(map(int, input("Enter the area values 'make sure its the same numer of the images': ").split()))
#area = ["", "", "", "", "", "", "", "", ""]  #write here the areas (must be the same number of images)
oldNew = list(map(str, input("is the leak old/new leak 'make sure its the same numer of the images': ").split()))

#Dictionary of leak Images
keys1 = "NO", "Name", "Location", "Image", "Area", "OldNew"
subject2 = NO_leak,name_leak,leak_images_location, predict_leak, area , oldNew
leak_dic = {z[0]:list(z[1:]) for z in zip(keys1* len(subject2),subject2)}
#print (leak_dic)


    
#---------------------------------------------------------------------------------------------------------------------#    
#---------------------------------------------------------------------------------------------------------------------#   
# Creating the PDF using ReportLab


from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import Image, Paragraph, SimpleDocTemplate, Table
from reportlab.lib.styles import getSampleStyleSheet

doc = SimpleDocTemplate("C:\\Users\\sami\\Desktop\\test images\\report\\Anomalies Detected.pdf", pagesize=letter) #change this path
#container for the 'Flowable' objects
elements = []
 

#Table 1
Ip = Image('pdo.jpg')
Ip.drawHeight = 1.25*inch*Ip.drawHeight / Ip.drawWidth
Ip.drawWidth = 1.25*inch

Ih = Image('haitch.png')
Ih.drawHeight = 1.25*inch*Ih.drawHeight / Ih.drawWidth
Ih.drawWidth = 1.25*inch


data1= [[Ip,'"Flowiness Anomalies Report"', Ih],[  ]]

 
t1=Table(data1,style=[('ALIGN',(0,0),(-1,-1),'CENTER'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('FONTSIZE', (0, 0), (-1, -1), 15),
                    ('BOX',(0,0),(-1,0),0.5,colors.black),
                    ('GRID',(0,0), (-1,0), 0.5,colors.black),
                    ])
t1._argW[2]=1.7*inch
t1._argW[1]=3.5*inch
t1._argW[0]=1.7*inch
 
elements.append(t1)



#Extract Info from User:
numFlights = input("Type the number of flights: ")
#print("Your Number of Flights is: ", numFlights)

reportID = input("Type the Report ID: ")

#Table 2
data2= [["Date", onlyDate],
       ["Location", " ???"],
       ["Method of Inspection", "UAS"],
       ["Methos of Analysis", "AI"],
       ["Number of flights", numFlights+" Flights"], #change Number of Flights
       ["Number of collected Images", (str(No_images)+" Images")],
       ["Flight Altitude Range", "50 - 100 m"],
       ["Report ID", reportID]] # Change
 
t2=Table(data2,style=[('ALIGN',(0,0),(-1,-1),'LEFT'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('BOX',(0,0),(-1,-1),0.5,colors.black),
                    ('GRID',(0,0), (-1,-1), 0.5,colors.black),
                    
])
t2._argW[1]=3.5*inch
t2._argW[0]=3.5*inch

elements.append(t2)

styleP = getSampleStyleSheet()['Normal']
#style.leading = 24
#style.fontName = 'Helvetica'
#style.alignment = TA_JUSTIFY
styleP.spaceAfter = 24

space1 = Paragraph('''''',styleP)
elements.append(space1)


#Table 3

def table3(no, location, area, oldNew, img):
    
    
    #area = "???" # Change area here
    #oldNew = "???" # change this
    
    data3= [["No","Anomaly", "Location Coordinates", "Estimated Area", "Old/New leak"],
           [str(no), " Oil Spill",str(location), area, oldNew],
           [ "","" ,img,"",""]]

 
    t3=Table(data3,style=[('ALIGN',(0,0),(-1,-1),'CENTER'),
                    ('VALIGN',(0,0),(-1,-1),'MIDDLE'),
                    ('BOX',(0,0),(-1,-2),0.5,colors.black),
                    ('GRID',(0,0), (-1,-2), 0.5,colors.black),
                    ])
    t3._argW[4]=1.5*inch
    t3._argW[3]=1.0*inch
    t3._argW[2]=3.0*inch
    t3._argW[1]=1.0*inch
    t3._argW[0]=0.5*inch
    
    return t3

#Repeat Table 3 from Leak Dictionary

for i in range(len(NO_leak)):
    Img = Image(leak_dic["Image"][0][i])
    Img.drawHeight = 3.25*inch*Ip.drawHeight / Ip.drawWidth
    Img.drawWidth = 3.25*inch
    
    elements.append(table3(leak_dic["NO"][0][i],
                           leak_dic["Location"][0][i],
                           leak_dic["Area"][0][i],
                           leak_dic["OldNew"][0][i],
                           Img))
    space2 = Paragraph('''''', styleP)
    elements.append(space2)


# write the document to disk
doc.build(elements)


#Print Duration Time of Model
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

{"message":"success","result":[{"message":"success","input":"20200114_0950_Lek_A5_Flight_01_00007.JPG","prediction":[]}]}
{"message":"success","result":[{"message":"success","input":"20200114_0950_Lek_A5_Flight_01_00014.JPG","prediction":[{"label":"Oil_Spill","xmin":935,"ymin":3748,"xmax":1935,"ymax":5287,"score":0.8830768},{"label":"Oil_Spill","xmin":959,"ymin":3531,"xmax":3110,"ymax":5247,"score":0.81634825}]}]}
{"message":"success","result":[{"message":"success","input":"20200114_0950_Lek_A5_Flight_01_00042.JPG","prediction":[{"label":"Oil_Spill","xmin":291,"ymin":56,"xmax":1990,"ymax":2044,"score":0.87230736}]}]}
{"message":"success","result":[{"message":"success","input":"Lekhwair_A_14012020_OilLeak_01,305350.JPG","prediction":[{"label":"Oil_Spill","xmin":1436,"ymin":3631,"xmax":3249,"ymax":5304,"score":0.9719354},{"label":"Oil_Spill","xmin":4250,"ymin":643,"xmax":5657,"ymax":2971,"score":0.7751789}]}]}
{"message":"success","result":[{"message":"success","input":"Lekhwair_A_140120